In [1]:
import pandas as pd 
import numpy as np
import os
from subprocess import check_output
import shutil

from rule import *
from amie import *

In [2]:
root_source = "./../../FB15k-237-num/"
store_data = "./../../Data_FB/FB15K_ST.tsv"

In [3]:
f = open(root_source+"FB15K_NumericalTriples.txt", "r")
dic_predicate = {}
for line in f:
    line_split = line.split("\n")[0].split("\t")
    line_split[2] = float(line_split[2]) 
    line_split = tuple(line_split)
    if line_split[1] in dic_predicate.keys():
        dic_predicate[line_split[1]].add(line_split)
    else : 
        dic_predicate[line_split[1]] = {line_split}
f.close()

In [4]:
thresholds = np.arange(0.25,1,0.25) #[0.25, 0.50, 0.75]
thresholds_str = ["-"+(str(int(i*100))) for i in thresholds]

In [5]:
def give_group(value, groups):
    for i, value_group in enumerate(groups):
        if value < value_group:
            return i
    return i+1

def write_file(X, f, groups):
    f.write(f"{X['Subject']}\t{X['Predicate']}\t{give_group(X['Object'], groups)}\n")

In [6]:
f = open(store_data, "w")

for key in dic_predicate.keys():
    tp_df = pd.DataFrame.from_dict(dic_predicate[key]).rename(columns={0: "Subject", 1: "Predicate", 2: "Object"})
    tp_df_describe = tp_df["Object"].quantile(thresholds)
    tp_df.apply(write_file, args=(f, tp_df_describe), axis=1)

f.close()

In [7]:
data = open(root_source+"train.txt", "r")
f = open(store_data, "a")

predicate_to_find = set()

for line in data: 
    f.write(line)
    predicate_to_find.add(line.split("\n")[0].split("\t")[1])
    
data.close()
f.close()

In [8]:
htr = ""
for p in predicate_to_find:
    htr+=p+","
htr = htr[:-1]
print(htr)

/film/film/distributors./film/film_film_distributor_relationship/film_distribution_medium,/location/location/time_zones,/people/deceased_person/place_of_burial,/olympics/olympic_participating_country/medals_won./olympics/olympic_medal_honor/olympics,/base/aareas/schema/administrative_area/capital,/tv/tv_producer/programs_produced./tv/tv_producer_term/program,/film/film/other_crew./film/film_crew_gig/film_crew_role,/film/film/country,/award/award_winning_work/awards_won./award/award_honor/award,/sports/professional_sports_team/draft_picks./sports/sports_league_draft_pick/school,/education/educational_institution_campus/educational_institution,/government/governmental_body/members./government/government_position_held/legislative_sessions,/education/university/fraternities_and_sororities,/base/schemastaging/person_extra/net_worth./measurement_unit/dated_money_value/currency,/film/film/produced_by,/base/locations/continents/countries_within,/people/person/place_of_birth,/film/film/other_cr

In [9]:
res = check_output(f'java -jar ./../amie3.jar -htr {htr} {store_data}', shell=True)

res_parsed = parse_amie(res)

In [10]:
len(res_parsed)

8010

In [11]:
res_parsed

{?b /music/instrument/family ?a => ?a /music/performance_role/guest_performances./music/recording_contribution/performance_role ?b,
 ?f /award/award_nominated_work/award_nominations./award/award_nomination/nominated_for ?a & ?b /award/award_nominated_work/award_nominations./award/award_nomination/nominated_for ?f => ?a /award/award_nominated_work/award_nominations./award/award_nomination/nominated_for ?b,
 ?b /base/aareas/schema/administrative_area/administrative_parent ?f & ?a /location/capital_of_administrative_division/capital_of./location/administrative_division_capital_relationship/administrative_division ?f => ?a /location/capital_of_administrative_division/capital_of./location/administrative_division_capital_relationship/administrative_division ?b,
 ?a /location/capital_of_administrative_division/capital_of./location/administrative_division_capital_relationship/administrative_division ?f & ?f /location/statistical_region/gdp_nominal./measurement_unit/dated_money_value/currency ?